<h1 style="text-align: center;">Azure DevOps Comments Monitor</h1>
<p style="text-align: center;"><em>Powered by Google ADK &amp; Azure DevOps MCP</em></p>

**This agent automatically monitors Azure DevOps work items for mentions of specific team members and delivers daily email summaries.** 

The agent works with any Azure DevOps organization and is perfect for engineering managers, team leads, and individual contributors who need to stay informed about discussions, decisions, and action items where they're mentioned.

### How It Works

1. **Query Work Items** - Retrieves work items from a saved Azure DevOps query
2. **Fetch Comments** - Collects all comments from each work item
3. **Filter & Process** - Identifies mentions of the target user within the last 2 days
4. **Format Report** - Creates a structured summary with work item context and full comment text
5. **Send Notification** - Delivers the formatted report via email


### Prerequisites

To run the Azure DevOps Mentions Monitor agent with an MCP connection, you need:

* **Python 3.10+** with a virtual environment (venv)
* **JupyterLab** (installed inside the venv) or Google Colab
* **Node.js** - includes npx, which is used to launch the Azure DevOps MCP server (@azure-devops/mcp) on the fly without a global install
* **Azure DevOps Access** - Valid Personal Access Token (PAT) with Work Items (Read) permissions
* **Email Credentials** - Gmail account with App Password for sending notifications
* **Environment Variables** configured in `.env` file:
  * `GOOGLE_API_KEY` - Gemini API key for the agent's LLM
  * `EMAIL_USER` - Gmail address for sending notifications
  * `EMAIL_PASS` - Gmail App Password
  * `TO_EMAIL` - Recipient email address for summaries


### Customization

You can easily customize the agent by modifying:

* **Target User** - Change `@Jenya Stoeva` to any user's display name
* **Time Window** - Adjust the 2-day filter to any desired timeframe
* **Query ID** - Point to different saved queries in your Azure DevOps organization
* **Email Format** - Modify the email template and subject line
* **Organization/Project** - Configure for any Azure DevOps organization and project

In [ ]:
%pip install google-adk mcp

In [1]:
import asyncio
import contextlib
import os
from datetime import datetime
from typing import Optional

from dotenv import load_dotenv
from google.genai import types
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools.mcp_tool.mcp_toolset import McpToolset, StdioConnectionParams, StdioServerParameters
from google.adk.tools.tool_context import ToolContext

In [2]:
load_dotenv()
GEMINI_API_KEY = os.getenv("GOOGLE_API_KEY")
to_email = os.getenv("TO_EMAIL")  # Your email address
from_email = os.getenv("EMAIL_USER")  # Gmail address
password = os.getenv("EMAIL_PASS")  # Gmail app password

if GEMINI_API_KEY:
    print("Google API key loaded!")
else:
    print("Google API key missing!")

Google API key loaded!


In [3]:
import smtplib
from email.mime.text import MIMEText

def send_devops_comments_summary_email(
    summary: str,
    tool_context: Optional[ToolContext] = None
) -> str:
    """Send email notification with Azure DevOps mentions summary."""
    
    if not all([from_email, password, to_email]):
        return "❌ Email credentials not configured. Set EMAIL_USER, EMAIL_PASS, and TO_EMAIL."
    
    try:
        current_date = datetime.now().strftime("%Y-%m-%d %H:%M")
        subject = f"[Azure DevOps] Daily Mentions Summary - {current_date}"
        
        body = f"""
Azure DevOps Mentions Summary
{'=' * 60}
Generated: {current_date}
            
{summary}
            
{'=' * 60}
This is an automated notification from your Azure DevOps Assistant.
            """
        
        msg = MIMEText(body, "plain")
        msg["Subject"] = subject
        msg["From"] = from_email
        msg["To"] = to_email
        
        with smtplib.SMTP_SSL("smtp.gmail.com", 465) as smtp:
            smtp.login(from_email, password)
            smtp.send_message(msg)
        
        return f"✅ Email sent to {to_email}"
        
    except Exception as e:
        return f"❌ Failed to send email: {str(e)}"

In [4]:
agent = Agent(
    name="azure_devops_assistant",
    model="gemini-2.5-flash",
    description="Azure DevOps assistant for monitoring specific person mentions in work items",
    instruction="""
            You are an Azure DevOps assistant that retrieves and summarizes recent comments mentioning @Jenya Stoeva.
            
            **4-STEP WORKFLOW - EXECUTE IN ORDER:**
            
            ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
            
            **STEP 1: GET WORK ITEMS**
            
            Call `wit_get_query_results_by_id` with:
            {
              "id": "2f7d034a-9d9d-4dce-97ff-b617811c82ac",
              "project": "MARMIND",
              "organization": "UPPER-Network"
            }
            
            Extract all work item IDs → Store as `work_item_ids`
            
            If this fails: Stop and report error.
            
            ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
            
            **STEP 2: GET ALL COMMENTS**
            
            For each ID in `work_item_ids`:
            - Get work item title
            - Call `wit_list_work_item_comments`
            - Store ALL comments with metadata (text, author, date, work item ID & title)
            
            Continue processing other items if one fails.
            
            ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
            
            **STEP 3: FILTER & FORMAT**
            
            Apply filters IN ORDER:
            1. Date: ONLY last 2 days from {current_date}
            2. Mention: ONLY comments containing "@Jenya Stoeva"
            3. Exclude work items with no qualifying comments
            
            Clean HTML:
            - Strip all tags (<div>, <a>, <br>, etc.)
            - Convert `<a href="#" data-vss-mention="...">@Name</a>` to `@Name`
            
            Format output as `formatted_summary`:
            
            **IF NO QUALIFYING COMMENTS:**
            "No mentions of @Jenya Stoeva in comments from the last 2 days."
            
            **IF QUALIFYING COMMENTS FOUND:**
            ```
            📋 Summary of Recent Mentions (Last 2 Days)
            Total Work Items with Mentions: [X]
            
            📝 Work Item Summaries
            
            1️⃣ Work Item #[ID]: [Title]
            🔗 https://dev.azure.com/UPPER-Network/MARMIND/_workitems/edit/[ID]
            
            Summary: [2-3 sentence summary of what was discussed and why Jenya was mentioned]
            
            ---
            
            [Repeat for each work item with qualifying comments]
            
            ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
            
            📝 Detailed Comments (Full Text)
            
            1️⃣ Work Item #[ID]: [Title]
            🔗 https://dev.azure.com/UPPER-Network/MARMIND/_workitems/edit/[ID]
            
            Comment from [Author] on [Date]:
            > [COMPLETE comment text - never truncate or summarize]
            
            ---
            
            [If multiple comments in this work item, include ALL]
            
            ---
            
            [Repeat for ALL qualifying comments in ALL work items]
            ```
            
            Requirements:
            - Sort by most recent first
            - Same work items in both sections
            - Include FULL comment text in detailed section
            
            ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
            
            **STEP 4: SEND EMAIL** 🚨 MANDATORY - CANNOT SKIP
            
            Call `send_devops_comments_summary_email` with `summary=formatted_summary`
            
            Send even if no mentions found.
            
            After successful send, reply: "✅ Email notification sent!"
            
            ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
            
            **EXECUTION RULES:**
            - Complete steps 1→2→3→4 in order
            - Do not skip Step 4 under any circumstances
            - Step 4 executes regardless of whether mentions were found
            
            Begin immediately.
    """,
    tools=[
        send_devops_comments_summary_email,
        McpToolset(
            connection_params=StdioConnectionParams(
                server_params=StdioServerParameters(
                    command='npx',
                    args=[
                        '-y',
                        '@azure-devops/mcp',
                        'UPPER-Network'  # Your organization
                    ],
                ),
                timeout=120 
            )
        ),
    ],
)


# Setup session and runner  
session_service = InMemorySessionService()
runner = Runner(agent=agent, app_name="azure_devops_assistant", session_service=session_service)

# Global session setup
USER_ID = "user"
SESSION_ID = "azure_devops_conversation"
session_initialized = False


async def chat():
    """Start interactive conversation with the agent"""
    global session_initialized
    
    if not session_initialized:
        await session_service.create_session(
            app_name="azure_devops_assistant",
            user_id=USER_ID,
            session_id=SESSION_ID,
            state={'current_date': datetime.now().strftime("%Y-%m-%d")}  # Initialize state here!
        )
        
        session_initialized = True
        print("🤖 Hello! I'm your Azure DevOps assistant. Type 'quit', 'exit', 'stop' to end our chat.")
    
    
    while True:
        user_input = input("You: ").strip()
        
        if user_input.lower() in ['quit', 'exit', 'stop']:
            print("👋 Goodbye!")
            break
            
        if not user_input:
            continue

        # Update current_date in session state before each run
        session = await session_service.get_session(
            app_name="azure_devops_assistant",
            user_id=USER_ID,
            session_id=SESSION_ID
        )
        session.state['current_date'] = datetime.now().strftime("%Y-%m-%d")
            
        message = types.Content(role='user', parts=[types.Part(text=user_input)])
        print("Agent: ", end="", flush=True)
        
        with open(os.devnull, 'w') as f, contextlib.redirect_stderr(f):
            async for event in runner.run_async(
                user_id=USER_ID,
                session_id=SESSION_ID,
                new_message=message
            ):
                if event.content and event.content.parts:
                    for part in event.content.parts:
                        if part.text:
                            print(part.text, end="", flush=True)
        
        print("\n")

In [ ]:
await chat()